## CRAG(Corrective Retrieval Augmented Generation)

> 참고: https://wikidocs.net/270686
> 관련 논문: https://arxiv.org/pdf/2401.15884

CRAG는 RAG의 개선된 버전입니다. **RAG의 한계를 보완**하기 위해 개발된 기술로, 검색된 문서의 품질을 평가하고 필요시 수정하는 메커니즘을 추가했습니다.

### 주요 특징

**1. 검색 평가자(Retrieval Evaluator)**
- 검색된 문서의 관련성과 품질을 평가
- 문서가 충분히 정확하고 관련성이 있는지 판단

**2. 수정 메커니즘**
- 검색 결과가 부정확하거나 불충분할 경우 대응
- 추가 검색, 웹 검색, 또는 지식 정제 수행

**3. 동적 처리 방식**
검색 품질에 따라 세 가지 경로로 분기:
- **Correct**: 검색 결과가 좋을 경우 → 그대로 사용
- **Incorrect**: 검색 결과가 나쁠 경우 → 웹 검색 등 추가 소스 활용
- **Ambiguous**: 애매한 경우 → 지식 정제 후 사용

### RAG와의 차이점

- **RAG**: 검색된 문서를 무조건 신뢰하고 사용
- **CRAG**: 검색 품질을 평가하고, 필요시 보완하거나 다른 소스 활용

이를 통해 더 정확하고 신뢰할 수 있는 답변 생성이 가능합니다.